# Reconhecimento de texto com EasyOCR

- Mantida por por [Jaided AI](https://jaided.ai/), uma empresa especializada em serviços de OCR
- Repositório oficial: https://github.com/JaidedAI/EasyOCR

In [ ]:
# Baixar imagens para o laboratório.
!wget https://github.com/fabiobento/ocr-sis-emb-2024-2/raw/refs/heads/main/imagens.zip
!unzip -n -q imagens.zip

# Baixar fontes para o laboratório
!wget https://github.com/fabiobento/ocr-sis-emb-2024-2/raw/refs/heads/main/fontes.zip
!unzip -n -q fontes.zip

# Instalação

In [ ]:
!pip install easyocr

In [ ]:
# Código necessário após atualização do opencv-python-headless no Colab para a versão 4.5.5.62, que não suporta o "_registerMatType".
!pip uninstall opencv-python-headless
!pip install opencv-python-headless==4.1.2.30

In [ ]:
from easyocr import Reader
import cv2
from google.colab.patches import imshow
from PIL import ImageFont, ImageDraw, Image
import numpy as np

# Configurações

In [ ]:
lista_idiomas = 'en,pt'
idiomas = lista_idiomas.split(',')
print(idiomas)

In [ ]:
gpu = True #@param {type: 'boolean'}

In [ ]:
img = cv2.imread('caneca.jpg')
imshow(img)

In [ ]:
original = img.copy()

# Reconhecimento (OCR)

In [ ]:
reader = Reader(idiomas, gpu)
resultados = reader.readtext(img)

In [ ]:
resultados

In [ ]:
resultados[0][0]

# Escrevendo os resultados na imagem

## Texto

In [ ]:
fonte = './Fontes/calibri.ttf'

In [ ]:
def escreve_texto(texto, x, y, img, fonte, cor=(50,50,255), tamanho = 22):
  fonte = ImageFont.truetype(fonte, tamanho)
  img_pil = Image.fromarray(img)
  draw = ImageDraw.Draw(img_pil)
  draw.text((x, y - tamanho), texto, font = fonte, fill = cor)
  img = np.array(img_pil)
  return img

## Caixa delimitadora

Cantos da bounding box:
* **te** = topo esquerdo (superior esquerdo)
* **td** = topo direito (superior direito)
* **bd** = baixo direito (inferior direito)
* **be** = baixo esquerdo (inferior esquerdo)

In [ ]:
def coordenada_caixa(caixa):
  (te, td, bd, be) = caixa
  te = (int(te[0]), int(te[1]))
  td = (int(td[0]), int(td[1]))
  bd = (int(bd[0]), int(bd[1]))
  be = (int(be[0]), int(be[1]))
  return te, td, bd, be

In [ ]:
coordenada_caixa(resultados[0][0])

In [ ]:
def desenha_caixa(img, te, bd, cor_caixa=(200,255,0), espessura=2):
  cv2.rectangle(img, te, bd, cor_caixa, espessura)
  return img

In [ ]:
img = original.copy()
for (caixa, texto, probabilidade) in resultados:
  #print(caixa, texto, probabilidade)
  te, td, bd, be = coordenada_caixa(caixa)
  img = desenha_caixa(img, te, bd)
  img = escreve_texto(texto, te[0], te[1], img, fonte)
imshow(img)

In [ ]:
img = cv2.imread('./Imagens/google-cloud.jpg')
reader = Reader(idiomas, gpu)
resultados = reader.readtext(img)

for (caixa, texto, probabilidade) in resultados:
  te, td, bd, be = coordenada_caixa(caixa)
  img = desenha_caixa(img, te, bd)
  img = escreve_texto(texto, te[0], te[1], img, fonte)
imshow(img)

# Outros idiomas

- Outros idiomas [veja aqui](https://www.jaided.ai/easyocr/)

In [ ]:
lista_idiomas = 'en,fr'
idiomas = lista_idiomas.split(',')
idiomas

In [ ]:
img = cv2.imread('./Imagens/frances.jpg')
reader = Reader(idiomas, gpu)
resultados = reader.readtext(img)
resultados

In [ ]:
for (caixa, texto, probabilidade) in resultados:
  te, td, bd, be = coordenada_caixa(caixa)
  img = desenha_caixa(img, te, bd)
  img = escreve_texto(texto, te[0], te[1], img, fonte)
imshow(img)

In [ ]:
lista_idiomas = 'en,ch_sim'
idiomas = lista_idiomas.split(',')
idiomas

In [ ]:
fonte = '/content/simsun.ttc'

In [ ]:
img = cv2.imread('./Imagens/chines.jpg')
reader = Reader(idiomas, gpu)
resultados = reader.readtext(img)
resultados

In [ ]:
for (caixa, texto, probabilidade) in resultados:
  te, td, bd, be = coordenada_caixa(caixa)
  img = desenha_caixa(img, te, bd)
  img = escreve_texto(texto, te[0], te[1], img, fonte)
imshow(img)

# Texto com fundo

In [ ]:
def fundo_texto(texto, x, y, img, fonte, tamanho=32, cor_fundo=(200,255,0)):
  fundo = np.full((img.shape), (0,0,0), dtype=np.uint8)
  texto_fundo = escreve_texto(texto, x, y, fundo, fonte, tamanho=tamanho)
  texto_fundo = cv2.dilate(texto_fundo,(np.ones((3,5),np.uint8)))
  fx,fy,fw,fh = cv2.boundingRect(texto_fundo[:,:,2])
  cv2.rectangle(img, (fx, fy), (fx + fw, fy + fh), cor_fundo, -1)
  return img

In [ ]:
fonte = './Fontes/calibri.ttf'
lista_idiomas = 'en,pt'
idiomas = lista_idiomas.split(',')

In [ ]:
img = cv2.imread('./Imagens/placa-infos.jpg')
reader = Reader(idiomas, gpu)
resultados = reader.readtext(img)
resultados

In [ ]:
cor_fonte = (0,0,0)
cor_fundo = (200,255,0)
cor_caixa = (200,255,0)
tamanho_fonte = 18

In [ ]:
for (caixa, texto, probabilidade) in resultados:
  te, td, bd, be = coordenada_caixa(caixa)
  img = desenha_caixa(img, te, bd, cor_caixa)
  img = fundo_texto(texto, te[0], te[1], img, fonte, tamanho_fonte, cor_fundo)
  img = escreve_texto(texto, te[0], te[1], img, fonte, cor_fonte, tamanho_fonte)
imshow(img)